<a href="https://colab.research.google.com/github/ldsAS/Tibame-AI-Learning/blob/main/Tibame20250612_%E9%9B%86%E6%88%90%E6%96%B9%E6%B3%95_Ensemble_Method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ensemble Method（集成方法）

Ensemble Method（集成方法）是機器學習中的一種技術，旨在通過結合多個基礎模型（稱為弱學習器）來提高整體的預測性能。這些基礎模型通常是同質的，即它們使用相同的學習算法，或者是異質的，即使用不同的學習算法。集成方法通過結合多個模型的預測結果，從而減少過擬合的風險、提高準確度，並增加穩定性。

## 常見的集成方法包括：

- Bagging（Bootstrap Aggregating）
    - 這種方法通過隨機抽樣的方式創建多個訓練集，並訓練多個模型，最終的預測結果通常是這些模型預測的平均或投票結果。
    - **隨機森林（Random Forest）** 是 Bagging 方法的一個例子。  
- Boosting
    - Boosting 旨在通過逐步構建一系列模型，其中每一個後續模型都試圖修正前一個模型的錯誤。最終的預測結果是所有模型預測的加權平均。
    - 常見的 Boosting 算法包括 **AdaBoost**、**Gradient Boosting**、**XGBoost** 等。  
- Stacking（堆疊）
    - Stacking 是將多個不同類型的模型的預測結果作為特徵，並通過另一個模型（通常稱為 meta-model）進行最終預測的過程。這種方法可以有效地融合不同模型的優勢。

## 優點

- **提高準確度**：
    通過集成多個模型，能夠減少單一模型的偏差和方差，提高預測的準確度。
- **減少過擬合**：
    特別是在 Bagging 和 Boosting 方法中，通過結合多個模型的預測結果，減少單一模型過擬合的風險。
- **穩定性更高**：
    集成方法比單一模型對噪聲和數據變異的敏感度低，通常具有更高的穩定性。
- **處理複雜問題**：
    通過使用多個不同的學習器來解決複雜的預測問題，能夠利用每個模型的優勢。

## 缺點
- **計算成本較高**：
    訓練多個模型會增加計算開銷和訓練時間，特別是在大規模數據集上。
- **解釋性差**：
    集成方法通常難以解釋，因為多個模型的結果是綜合起來的，很難直接理解模型的內部決策過程。
- **需要更多的內存和存儲空間**：
    需要存儲多個模型，對於資源有限的情況可能不適用。
- **難以調參**：
    集成方法通常涉及多個模型，每個模型都可能有不同的超參數，調整這些參數可能較為繁瑣。

集成方法能夠有效地提高預測性能，減少過擬合的風險，但需要更多的計算資源並且缺乏解釋性。它在許多實際應用中都能發揮重要作用，特別是在需要高準確度的問題領域。


## 載入資料

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data = pd.read_csv("../data/R公司_人員資料_train.csv")

input_field = ['性別(Gender)', '年齡(Age)', '是否為成年人(Over18)',
       '婚姻狀態(MaritalStatus)', '通勤距離(DistanceFromHome)', '教育程度(Education)',
       '教育專業領域(EducationField)', '部門(Department)', '職位名稱(JobRole)',
       '職位等級(JobLevel)', '在該公司工作總年資(YearsAtCompany)',
       '在該職位工作年資(YearsInCurrentRole)', '在該職等工作年資(YearsSinceLastPromotion)',
       '與現任管理者工作年資(YearsWithCurrManager)', '總工作年資(TotalWorkingYears)',
       '過去工作公司家數(NumCompaniesWorked)', '每日工資額(DailyRate)',
       '每小時工資額(HourlyRate)', '月收入(MonthlyIncome)', '月費率(MonthlyRate)',
       '標準工作時間(StandardHours)', '加班(OverTime)', '調薪百分比(PercentSalaryHike)',
       '股票選擇權等級(StockOptionLevel)', '績效評估(PerformanceRating)',
       '出差頻率(BusinessTravel)', '去年訓練時間(TrainingTimesLastYear)',
       '工作滿意度(JobSatisfaction)', '工作環境滿意度(EnvironmentSatisfaction)',
       '工作投入(JobInvolvement)', '工作生活平衡(WorkLifeBalance)',
       '人際關係滿意度(RelationshipSatisfaction)']
target_field = '離職(Attrition)'
target_filed_map = {"Yes": 1, "No": 0}

data_input = data[input_field]
data_target = data[target_field].map(target_filed_map)

data_input = pd.get_dummies(data_input)

In [ ]:
from sklearn.model_selection import cross_validate
def run_cv(model):
    score_type = ['balanced_accuracy','roc_auc']
    cv_scores = cross_validate(model, data_input, data_target, cv=5, n_jobs=5, return_train_score=True, scoring=score_type)
    for key in cv_scores:
        print(key,' : ', cv_scores[key].mean())

# Decision Tree
Base Line

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()

run_cv(model)

fit_time  :  0.011723041534423828
score_time  :  0.00440821647644043
test_balanced_accuracy  :  0.5987009419152276
train_balanced_accuracy  :  1.0
test_roc_auc  :  0.5987009419152276
train_roc_auc  :  1.0


# Bagging
**Bagging**（Bootstrap Aggregating）是一種集成學習（Ensemble Learning）技術，旨在通過將多個模型的預測結果進行結合來提高預測的準確性和穩定性。它的基本思想是通過對訓練資料進行重複抽樣，生成多個訓練子集，並基於這些子集訓練多個模型，最後將這些模型的預測結果進行結合，從而減少過擬合的風險。

## Bagging的主要特點：

1. **重複抽樣（Bootstrap Sampling）**
   - Bagging通過對原始訓練資料進行有放回的隨機抽樣，生成多個不同的訓練子集。每個子集可能包含重複的樣本，這樣可以增加訓練數據的多樣性。
2. **多模型訓練**
   - 在每個訓練子集上，訓練一個基礎模型。這些基礎模型通常是獨立且相同的，例如決策樹（也就是隨機森林中的基礎決策樹）。
3. **結果聚合**
   - 在預測階段，對所有基礎模型的預測結果進行聚合。對於分類問題，通常使用**投票法**（majority voting）；對於回歸問題，則使用**平均法**。
4. **減少過擬合**
   - 由於每個模型在不同的數據子集上訓練，這樣可以減少模型對特定訓練數據的過擬合，從而提升模型的泛化能力。

## Bagging的優勢：
- **降低方差**：Bagging減少了單個模型的方差，通常會得到更穩定的預測結果。
- **提高預測準確度**：通過集成多個模型的預測，Bagging能夠提高預測的準確度，尤其是對於高方差的基礎模型（如決策樹）。
- **並行化**：由於每個基礎模型都是獨立訓練的，因此Bagging算法可以輕鬆進行並行計算，從而提高訓練效率。

## 常見的Bagging算法：
- **隨機森林（Random Forest）**：隨機森林是Bagging的延伸，利用多棵隨機選擇特徵的決策樹來進行預測，進一步增加模型的多樣性並降低過擬合。


## BaggingClassifier

In [ ]:
from sklearn.ensemble import BaggingClassifier
model = BaggingClassifier(n_estimators=10)

run_cv(model)

fit_time  :  0.11075596809387207
score_time  :  0.011260986328125
test_balanced_accuracy  :  0.613770277341706
train_balanced_accuracy  :  0.9532561101823103
test_roc_auc  :  0.7556036760858189
train_roc_auc  :  0.9994733931552358


In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
model = BaggingClassifier(DecisionTreeClassifier())

run_cv(model)

fit_time  :  0.13799672126770018
score_time  :  0.014586639404296876
test_balanced_accuracy  :  0.6126648351648353
train_balanced_accuracy  :  0.960843085741581
test_roc_auc  :  0.7572474489795917
train_roc_auc  :  0.9995636953249118


## RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()

run_cv(model)

fit_time  :  0.33660049438476564
score_time  :  0.028206348419189453
test_balanced_accuracy  :  0.596361852433281
train_balanced_accuracy  :  1.0
test_roc_auc  :  0.8105625327053898
train_roc_auc  :  1.0


### RandomForest 內建欄位關鍵度評估

In [ ]:
model.fit(data_input, data_target)
importances = model.feature_importances_
feature_names = model.feature_names_in_

for i in range(0,len(importances)):
    print(feature_names[i], ' = ',importances[i])

年齡(Age)  =  0.056808296978834785
通勤距離(DistanceFromHome)  =  0.0405274073692964
教育程度(Education)  =  0.019360681915986905
職位等級(JobLevel)  =  0.02542443834946225
在該公司工作總年資(YearsAtCompany)  =  0.040434580813888066
在該職位工作年資(YearsInCurrentRole)  =  0.03278910941256539
在該職等工作年資(YearsSinceLastPromotion)  =  0.025887071381549772
與現任管理者工作年資(YearsWithCurrManager)  =  0.025941708470322452
總工作年資(TotalWorkingYears)  =  0.044637131697863576
過去工作公司家數(NumCompaniesWorked)  =  0.0339400203501874
每日工資額(DailyRate)  =  0.0436188772005599
每小時工資額(HourlyRate)  =  0.041361278878690025
月收入(MonthlyIncome)  =  0.08070266493671623
月費率(MonthlyRate)  =  0.04273308846444309
標準工作時間(StandardHours)  =  0.0
調薪百分比(PercentSalaryHike)  =  0.0304369841029324
股票選擇權等級(StockOptionLevel)  =  0.026101635039405154
績效評估(PerformanceRating)  =  0.0033256282336598776
去年訓練時間(TrainingTimesLastYear)  =  0.022876698187359185
工作滿意度(JobSatisfaction)  =  0.02544019761631164
工作環境滿意度(EnvironmentSatisfaction)  =  0.02409725059309701
工作投入(JobInvo

In [ ]:
#重要性的總合為1
importances.sum()

np.float64(0.9999999999999999)

# Boost
**Boost**（提升）是指一類用於提升模型性能的機器學習方法，通常以集成學習（Ensemble Learning）為基礎，通過將多個弱學習器結合成一個強學習器來提高預測準確度。Boosting方法會重點處理那些在前一輪中錯誤預測的樣本，從而使得模型對難以分類的樣本有更好的識別能力。

## Boost的主要特點：
1. **加強難以分類的樣本**
   - Boosting的基本思想是對於每一輪訓練，根據前一輪模型的預測結果調整樣本的權重。錯誤預測的樣本會被賦予更高的權重，這樣在下一輪訓練時，模型會更加關注這些錯誤樣本。
2. **基於弱學習器的集成**
   - Boosting通常使用弱學習器（例如簡單的決策樹）作為基礎學習器。這些弱學習器的表現可能較差，但通過加強和集成多個模型，最終能得到較強的預測能力。
3. **加法模型**
   - 在Boosting過程中，每一輪的模型預測結果會加到之前的模型上，最終的預測是所有基礎模型預測結果的加權和。
4. **強化學習的準確性**
   - 通過多輪迭代和加權，Boosting能夠逐漸提升模型的準確度，特別是在處理復雜或不平衡數據集時。

## Boost的常見算法：
1. **AdaBoost（Adaptive Boosting）**
   - AdaBoost是最早的Boosting算法之一。在每一輪中，它會根據前一輪錯誤的樣本，調整樣本的權重，使得下一輪模型更加關注這些錯誤的樣本。最終的預測是所有弱學習器預測結果的加權和。
2. **Gradient Boosting**
   - Gradient Boosting（梯度提升）是一種基於梯度下降的Boosting算法。它通過每次選擇一個新的模型來減少當前模型的誤差，這些模型是基於負梯度方向進行擬合的。
3. **XGBoost**
   - XGBoost是Gradient Boosting的一個高效實現，具有更多的優化和正則化選項，能處理缺失值並且能有效避免過擬合，是目前最流行的Boosting算法之一。
4. **LightGBM**
   - LightGBM（Light Gradient Boosting Machine）是一個高效的梯度提升算法，主要通過優化樹結構和採用直方圖算法來加速模型訓練，特別適合大規模數據。

## Boost的優勢：
- **提高準確度**：通過加強錯誤樣本的學習，Boosting方法通常能顯著提高模型的預測準確度。
- **有效處理不平衡數據**：Boosting能夠更好地處理類別不平衡的問題，尤其是當一些類別的樣本較少時。
- **強化模型的表現**：即使是表現不佳的基礎學習器，在Boosting的加持下，最終也能達到較高的預測效果。

## Boost的缺點：
- **過擬合風險**：如果模型訓練過長，Boosting可能會過擬合訓練數據，尤其是在數據噪聲較多的情況下。
- **計算成本較高**：由於Boosting需要多次迭代，並且每一輪模型的訓練是基於前一輪的錯誤，這會導致訓練時間較長。


## AdaBoost
**AdaBoost**（Adaptive Boosting）是一種集成學習方法，屬於Boosting家族的算法。它通過將多個弱學習器（通常是簡單的模型，如決策樹）結合成一個強學習器來提高預測性能。AdaBoost的核心思想是根據前一輪模型的錯誤，對錯誤預測的樣本給予更高的權重，並在後續的模型中更加關注這些樣本，從而提升模型的準確度。

### AdaBoost的工作原理：
1. **初始化樣本權重**：
   - 在訓練過程開始時，對所有訓練樣本賦予相同的權重。
2. **訓練弱學習器**：
   - 依次訓練多個弱學習器（例如簡單的決策樹）。每一輪的學習器會根據當前樣本的權重進行訓練。
3. **計算每個弱學習器的錯誤率**：
   - 計算每一輪學習器的錯誤率，即該學習器在當前加權樣本上的錯誤預測數量。
4. **更新樣本權重**：
   - 根據每個弱學習器的錯誤率，對錯誤分類的樣本賦予更高的權重，正確分類的樣本權重則降低。這樣，在下一輪訓練中，錯誤預測的樣本會得到更多的關注。
5. **加權集成預測**：
   - 最終的預測結果是每個弱學習器預測結果的加權和。每個學習器的權重與其準確性有關，錯誤較小的學習器權重較高。

### AdaBoost的特點：
1. **自適應性**：
   - AdaBoost根據前一輪的錯誤來自適應調整樣本權重，使模型更加關注難以分類的樣本。
2. **加強弱學習器**：
   - 儘管每個弱學習器的預測能力可能較差，但通過加權集成多個弱學習器，最終的預測結果可以顯著提高。
3. **簡單高效**：
   - AdaBoost可以使用相對簡單的基礎模型（如決策樹）來實現強大的預測效果。
4. **易於實現和理解**：
   - 算法結構簡單，且具有較好的解釋性，容易理解和實現。

### AdaBoost的優勢：
- **提高預測準確度**：通過多輪迭代和加權集成，AdaBoost能夠顯著提高模型的預測準確度，尤其是在弱學習器的基礎上。
- **減少過擬合風險**：與其他集成方法（如Bagging）相比，AdaBoost能夠集中學習錯誤樣本，減少過擬合的風險。
- **高效訓練**：AdaBoost能夠在相對較少的訓練輪次中達到較好的效果，訓練過程相對高效。

### AdaBoost的缺點：
- **對噪聲敏感**：AdaBoost可能會對噪聲或異常值過於敏感，因為錯誤的樣本權重會在後續的訓練中被不成比例地加重。
- **過度加強錯誤樣本**：如果訓練數據中有大量錯誤樣本或極端值，AdaBoost可能會過度關注這些樣本，導致過擬合。

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier()

run_cv(model)

fit_time  :  0.23138628005981446
score_time  :  0.03842940330505371
test_balanced_accuracy  :  0.6591313448456306
train_balanced_accuracy  :  0.7033550316259284
test_roc_auc  :  0.8331526687598115
train_roc_auc  :  0.9083435992736042


## Gradient Boosting
**Gradient Boosting**（梯度提升）是一種強大的集成學習方法，屬於Boosting家族。它通過結合多個弱學習器（通常是決策樹）來提升預測準確性。與其他Boosting算法不同，Gradient Boosting使用梯度下降法來逐步優化模型，將每一輪新模型的學習方向設置為當前模型誤差的負梯度。這樣逐步減少模型的預測誤差。

### Gradient Boosting的工作原理：
1. **初始化模型**：
   - 開始時，訓練一個基礎模型，通常是對所有樣本預測其均值（回歸問題）或最常見的類別（分類問題）。
2. **計算誤差（負梯度）**：
   - 計算當前模型的預測誤差，這些誤差是目標函數對模型的梯度（即負梯度）。對於回歸問題，這些誤差是實際值與預測值之間的差；對於分類問題，這些誤差是對數損失的梯度。
3. **擬合負梯度**：
   - 根據誤差（負梯度），訓練一個新的模型（通常是一棵決策樹），使其擬合這些負梯度。
4. **更新模型**：
   - 將新模型的預測加到前一輪的模型中，並根據一定的學習率（步長）來更新最終的預測模型。
5. **重複迭代**：
   - 重複這個過程多次，每次都訓練一個新的模型來擬合前一輪模型的誤差，並逐步改進模型的預測能力。
6. **最終預測**：
   - 經過多輪迭代後，最終的預測是所有基礎模型的加權和，這些加權值由學習率控制。

### Gradient Boosting的特點：
1. **逐步減少誤差**：
   - 每一輪的模型訓練都專注於最小化當前模型的誤差，這使得Gradient Boosting能夠逐步提升預測準確度。
2. **基於梯度下降**：
   - 利用梯度下降法，Gradient Boosting能夠在每一輪中找到最佳的擬合方向，從而有效地減少誤差。
3. **強大的模型擬合能力**：
   - 由於每個新的模型專注於擬合上一輪模型的誤差，Gradient Boosting能夠有效捕捉數據中的複雜模式，並具有較強的預測能力。
4. **可以處理不同類型的問題**：
   - 這種方法不僅能處理回歸問題，還能處理分類問題，並且具有良好的泛化能力。

### Gradient Boosting的優勢：
- **高準確度**：通過多次迭代和逐步減少誤差，Gradient Boosting通常能夠提供非常高的預測準確度。
- **靈活性**：它可以使用不同的基礎學習器（如決策樹），並且支持多種損失函數，使其可以應對各種回歸和分類問題。
- **強大的泛化能力**：經過適當的正則化，Gradient Boosting可以有效避免過擬合，並在大多數情況下具有較好的泛化能力。

### Gradient Boosting的缺點：
- **訓練時間較長**：由於每一輪的模型都是在前一輪模型的基礎上進行的，這使得Gradient Boosting的訓練過程比較耗時，特別是在大規模數據集上。
- **對噪聲敏感**：如果數據中包含大量的噪聲或異常值，Gradient Boosting可能會過擬合這些噪聲，從而影響模型的預測效果。
- **需要調整超參數**：為了獲得最佳性能，Gradient Boosting通常需要進行較為細致的超參數調整（例如學習率、樹的深度等）。

### 常見的Gradient Boosting實現：
1. **XGBoost**：
   - XGBoost是Gradient Boosting的高效實現，具有許多優化，如支持並行計算、正則化等。它已經成為機器學習中最常用的算法之一。
2. **LightGBM**：
   - LightGBM是一個快速、分佈式的Gradient Boosting實現，特別適合大規模數據集。它使用直方圖算法來加速訓練，並且能夠有效處理大數據。
3. **CatBoost**：
   - CatBoost是另一個高效的Gradient Boosting實現，專注於提高類別特徵的處理能力。它對類別特徵的處理比XGBoost和LightGBM更為高效。

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier()

run_cv(model)

fit_time  :  0.5595423221588135
score_time  :  0.008278608322143555
test_balanced_accuracy  :  0.6446546310832025
train_balanced_accuracy  :  0.9127375038166239
test_roc_auc  :  0.8207005494505493
train_roc_auc  :  0.9916805982883918


# XGBoost
* https://xgboost.readthedocs.io/

XGBoost（Extreme Gradient Boosting）是一種高效的機器學習算法，屬於梯度提升（Gradient Boosting）框架的擴展。它主要用於分類和回歸問題，因其在許多資料科學競賽中表現優異而廣受歡迎。

## XGBoost的主要特點：

1. **梯度提升（Gradient Boosting）**
   - XGBoost通過組合多個弱學習器（通常是決策樹）來構建一個強學習器。每一個新模型都會試圖最小化前一個模型的誤差。
2. **正則化**
   - XGBoost加入了L1（Lasso）和L2（Ridge）正則化項，能有效避免過擬合，提升泛化能力。
3. **處理缺失值**
   - XGBoost具有自動處理缺失資料的能力，並且在訓練過程中能夠對缺失值進行適當的處理。
4. **並行計算**
   - XGBoost採用了並行計算來加速模型訓練，特別是在特徵選擇和樹結構優化方面，能顯著提高效率。
5. **支持多種損失函數**
   - XGBoost支持多種損失函數（如平方誤差、對數損失等），可以根據不同的任務選擇合適的損失函數。
6. **交叉驗證和早停法**
   - XGBoost提供了交叉驗證和早停法功能，幫助在訓練過程中選擇最佳的超參數並防止過擬合。
7. **高效的樹結構優化**
   - XGBoost採用了更高效的算法進行樹結構的分裂和優化，能進一步提高模型訓練的速度。

由於這些特點，XGBoost被廣泛應用於各類資料分析任務，尤其在競賽、工業應用和大規模資料處理方面表現優異。


In [ ]:
# ! pip install xgboost

In [ ]:
from xgboost import XGBClassifier

model = XGBClassifier(n_estimators=100, learning_rate= 0.3)
model = xgb.XGBClassifier(subsample=0.8)

model.fit(data_input, data_target)
# 使用訓練資料預測分類
predicted = model.predict(data_input)
predicted

array([0, 0, 0, ..., 1, 0, 0], shape=(1176,))

因為xgboost並非scikit-learn的內建套件，所以不完全相容於scikit-learn的功能

譬如 cross_validate() 就無法正常執行

```python
score_type = ['balanced_accuracy','roc_auc']
cv_scores = cross_validate(model, data_input, data_target, cv=5, n_jobs=5, return_train_score=True, scoring=score_type)

In [ ]:
import xgboost as xgb
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 創建 XGBoost DMatrix，這是 XGBoost 特有的數據結構
train_data = xgb.DMatrix(data_input, label=data_target)

# 設置模型參數
params = {
    'objective': 'multi:softmax',  # 目標是多類分類
    'num_class': 3,  # 有三個類別
    'max_depth': 3,  # 決策樹的最大深度
    'eta': 0.1,  # 學習率
    'eval_metric': 'merror'  # 用來衡量模型的準確度
}

# 設置交叉驗證的參數
num_round = 100  # 訓練的迭代次數
cv_results = xgb.cv(
    params,               # 參數設置
    train_data,           # 訓練數據
    num_round,            # 訓練輪數
    nfold=5,              # 進行 5-fold 交叉驗證
    metrics='merror',     # 使用錯誤率 (misclassification error) 作為評估指標
    early_stopping_rounds=10,  # 早停策略，10輪內如果性能未提升則停止訓練
    as_pandas=True,       # 返回 pandas DataFrame 格式的結果
    seed=42               # 隨機種子
)

# 顯示交叉驗證結果
print(cv_results)

# 找出最佳的迭代次數
best_iteration = cv_results['test-merror-mean'].idxmin()
print(f"Best iteration: {best_iteration}")


    train-merror-mean  train-merror-std  test-merror-mean  test-merror-std
0            0.134141          0.003926          0.163264         0.015108
1            0.131590          0.004105          0.156459         0.014547
2            0.131590          0.005138          0.157310         0.015188
3            0.128827          0.003668          0.150501         0.015505
4            0.126700          0.003876          0.155615         0.015153
5            0.126275          0.005818          0.153913         0.014098
6            0.123724          0.004873          0.148810         0.014987
7            0.122661          0.005785          0.145406         0.016213
8            0.121598          0.005427          0.145398         0.011409
9            0.120961          0.005093          0.147101         0.014798
10           0.119472          0.006208          0.143696         0.012306
11           0.119048          0.006197          0.142849         0.011495
12           0.118622    

# Stacking

**Stacking**（堆疊）是一種集成學習方法，旨在將多個不同的基礎學習器結合起來，通過將各個模型的預測結果作為新特徵來訓練最終的預測模型。這種方法通常會用到兩層模型：第一層由多個基礎學習器組成，第二層則是學習如何將第一層的預測結果結合起來。

## Stacking的工作原理：

1. **基礎學習器訓練**：
   - 在第一層，選擇多個不同的基礎學習器（如決策樹、支持向量機、邏輯回歸等），並用相同的訓練數據訓練這些模型。每個基礎學習器會獨立預測訓練樣本。
2. **生成新的訓練數據**：
   - 將每個基礎學習器的預測結果（對於每個樣本）作為新的特徵，並構成新的訓練數據集。這些新特徵將成為第二層模型的輸入。
3. **第二層模型訓練**：
   - 使用新生成的特徵來訓練第二層的學習器（通常是線性模型，如邏輯回歸或簡單的線性回歸）。這一層模型學習如何根據第一層模型的預測結果來做出最終的預測。
4. **最終預測**：
   - 在預測階段，首先通過第一層的基礎學習器得到各自的預測結果，然後將這些結果作為新特徵輸入到第二層模型中，最終得出最終預測結果。

## Stacking的特點：
1. **多樣性**：
   - Stacking方法使用多種不同的基礎學習器，因此能夠結合它們的優勢，提高預測的穩定性和準確度。
2. **兩層結構**：
   - Stacking分為兩層：第一層由多個基礎學習器組成，第二層則是學習如何將這些基礎學習器的結果進行組合。
3. **靈活性**：
   - 可以根據不同的問題和需求選擇適合的基礎學習器和第二層模型，這使得Stacking具有很高的靈活性。
4. **非線性組合**：
   - 第二層的模型可以學習到基礎學習器預測結果之間的複雜關係，從而提供比單一模型更強的預測能力。

## Stacking的優勢：
- **提高準確度**：通過集成多個基礎學習器的預測，Stacking通常能夠提供比單一模型更高的預測準確度。
- **捕捉更多樣本模式**：多個基礎學習器能夠捕捉數據中的不同模式，從而提高模型的泛化能力。
- **減少過擬合**：由於第二層模型在學習如何組合基礎學習器的預測結果，Stacking有時能夠減少過擬合的風險。

## Stacking的缺點：
- **計算成本較高**：由於需要訓練多個基礎學習器以及第二層模型，Stacking的計算開銷通常比較大。
- **需要更多的數據**：為了訓練第二層模型，Stacking通常需要較多的數據，尤其是在基礎學習器較多的情況下。
- **較難調整**：由於Stacking的結構較為複雜，超參數調整可能需要更多的時間和精力。

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier


estimators_list=[
    ('Tree', DecisionTreeClassifier() ),
    ('KNN', KNeighborsClassifier() ),
    ('LogisticRegression', LogisticRegression() ),
]

model = StackingClassifier(
    estimators=estimators_list,
    final_estimator=SVC())

model

StackingClassifier(estimators=[('Tree', DecisionTreeClassifier()),
                               ('KNN', KNeighborsClassifier()),
                               ('LogisticRegression', LogisticRegression())],
                   final_estimator=SVC())